In [1]:
import os
from openai import OpenAI

## default

In [2]:
# chat
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    model="gpt-3.5-turbo",
    temperature=0.2
)
print(chat_completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


## image input

In [4]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    }
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].message.content)


This image features a serene landscape with a wooden boardwalk stretching through a vibrant green grassy field. The sky is beautifully clear with some scattered clouds, adding to the picturesque quality of the scene. The boardwalk leads the viewer's eye towards the horizon, suggesting a peaceful stroll in nature. The overall ambiance of the image is tranquil and inviting, perfect for those who appreciate outdoor scenery and the beauty of a natural setting.


## streaming

In [4]:
from openai import OpenAI

completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
        ],
    stream=True
)

for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


Hello! How can I help you today?

## Functions

In [20]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(completion.choices[0])


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nRsB6vo7QCevf088Bwv24s3y', function=Function(arguments='{"location":"Boston, MA","unit":"fahrenheit"}', name='get_current_weather'), type='function')]))


## Logprobs

In [5]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "user", "content": "Hello!"}
    ],
    logprobs=True,
    top_logprobs=2
)

print(completion.choices[0].message)
print(completion.choices[0].logprobs)


ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.00065310515, top_logprobs=[TopLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.00065310515), TopLogprob(token='Hi', bytes=[72, 105], logprob=-7.3495417)]), ChatCompletionTokenLogprob(token='!', bytes=[33], logprob=-1.1517961e-05, top_logprobs=[TopLogprob(token='!', bytes=[33], logprob=-1.1517961e-05), TopLogprob(token=' there', bytes=[32, 116, 104, 101, 114, 101], logprob=-11.396103)]), ChatCompletionTokenLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-0.00014942927, top_logprobs=[TopLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-0.00014942927), TopLogprob(token=' What', bytes=[32, 87, 104, 97, 116], logprob=-8.810857)]), ChatCompletionTokenLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=-4.3202e-07, top_logprobs=[T